# Semantic Kernel Multi-Agent Orchestration: Hands-On Exercise

In this notebook, you'll learn how to use the new Semantic Kernel agent orchestration framework in Python. We'll build a multi-agent discussion system step by step, explaining each part of the code and the framework's inner workings.

I noticed that some of the agents can get blocked by our content filter. If this is the case, create a custom filter
and assign it to your model deployment.

![](../../images/1_custom_filter.png)
![](../../images/2_custom_filter.png)

## 1. Install and Import Required Packages
We'll use the latest Semantic Kernel Python SDK and supporting libraries. Make sure you have installed the requirements:

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import standard libraries
import asyncio
import os
import sys

# Import dotenv to load environment variables from a .env file
import dotenv

# Import Semantic Kernel agent orchestration classes
from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.contents import AuthorRole, ChatHistory, ChatMessageContent
from semantic_kernel.functions import KernelArguments
from semantic_kernel.kernel import Kernel
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig

# Compatibility for Python <3.12
if sys.version_info >= (3, 12):
    from typing import override  # pragma: no cover
else:
    from typing_extensions import override  # pragma: no cover

# Load environment variables from .env in the notebook's directory
# This ensures .env is loaded even if you start Jupyter from a different folder
notebook_dir = os.path.dirname(os.path.abspath("semantic-kernel-agents-exercise.ipynb"))
dotenv.load_dotenv(dotenv_path=os.path.join(notebook_dir, ".env"))

True

## 2. Configure Azure OpenAI Chat Completion

We define a helper function to create an AzureChatCompletion service using environment variables. This allows us to easily switch between different Azure OpenAI deployments.

In [12]:
def get_azure_chat_completion():
    """Create AzureChatCompletion with config from environment variables."""
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
    if not all([endpoint, api_key, deployment]):
        raise ValueError("Please set AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, and AZURE_OPENAI_DEPLOYMENT in your .env file.")
    return AzureChatCompletion(
        endpoint=endpoint,
        api_key=api_key,
        deployment_name=deployment,
    )

## 3. Define the Agents

Each agent represents a unique perspective in the discussion. We use `ChatCompletionAgent` to define their roles, instructions, and connect them to the Azure OpenAI service.

In [6]:
def get_agents() -> list[Agent]:
    """Return a list of agents for the group discussion."""
    service = get_azure_chat_completion()
    agents = [
        ChatCompletionAgent(
            name="Farmer",
            description="A rural farmer from Southeast Asia.",
            instructions=(
                "You're a farmer from Southeast Asia. "
                "Your life is deeply connected to land and family. "
                "You value tradition and sustainability. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Developer",
            description="An urban software developer from the United States.",
            instructions=(
                "You're a software developer from the United States. "
                "Your life is fast-paced and technology-driven. "
                "You value innovation, freedom, and work-life balance. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Teacher",
            description="A retired history teacher from Eastern Europe",
            instructions=(
                "You're a retired history teacher from Eastern Europe. "
                "You bring historical and philosophical perspectives to discussions. "
                "You value legacy, learning, and cultural continuity. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Activist",
            description="A young activist from South America.",
            instructions=(
                "You're a young activist from South America. "
                "You focus on social justice, environmental rights, and generational change. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="SpiritualLeader",
            description="A spiritual leader from the Middle East.",
            instructions=(
                "You're a spiritual leader from the Middle East. "
                "You provide insights grounded in religion, morality, and community service. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Artist",
            description="An artist from Africa.",
            instructions=(
                "You're an artist from Africa. "
                "You view life through creative expression, storytelling, and collective memory. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Immigrant",
            description="An immigrant entrepreneur from Asia living in Canada.",
            instructions=(
                "You're an immigrant entrepreneur from Asia living in Canada. "
                "You balance trandition with adaption. "
                "You focus on family success, risk, and opportunity. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
        ChatCompletionAgent(
            name="Doctor",
            description="A doctor from Scandinavia.",
            instructions=(
                "You're a doctor from Scandinavia. "
                "Your perspective is shaped by public health, equity, and structured societal support. "
                "You are in a debate. Feel free to challenge the other participants with respect."
            ),
            service=service,
        ),
    ]
    return agents

## 4. Create a Custom Group Chat Manager

The `GroupChatManager` controls the flow of the conversation: when to end, who speaks next, and how to summarize. We subclass it to customize prompts and logic for our scenario.

In [7]:
class ChatCompletionGroupChatManager(GroupChatManager):
    """Custom group chat manager for orchestrating the discussion."""

    service: ChatCompletionClientBase
    topic: str

    # Prompts for the mediator agent
    termination_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to determine if the discussion has reached a conclusion. "
        "If you would like to end the discussion, please respond with True. Otherwise, respond with False."
    )
    selection_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to select the next participant to speak. "
        "Here are the names and descriptions of the participants: "
        "{{$participants}}\n"
        "Please respond with only the name of the participant you would like to select."
    )
    result_filter_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You have just concluded the discussion. "
        "Please summarize the discussion and provide a closing statement."
    )

    def __init__(self, topic: str, service: ChatCompletionClientBase, **kwargs) -> None:
        super().__init__(topic=topic, service=service, **kwargs)

    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Render a prompt with arguments using the kernel's template system."""
        prompt_template_config = PromptTemplateConfig(template=prompt)
        prompt_template = KernelPromptTemplate(prompt_template_config=prompt_template_config)
        return await prompt_template.render(Kernel(), arguments=arguments)

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        # This manager does not require user input between agent turns
        return BooleanResult(
            result=False,
            reason="This group chat manager does not require user input.",
        )

    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        # Ask the mediator if the discussion should end
        should_terminate = await super().should_terminate(chat_history)
        if should_terminate.result:
            return should_terminate
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.termination_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Determine if the discussion should end."),
        )
        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=BooleanResult),
        )
        termination_with_reason = BooleanResult.model_validate_json(response.content)
        print("*********************")
        print(f"Should terminate: {termination_with_reason.result}\nReason: {termination_with_reason.reason}.")
        print("*********************")
        return termination_with_reason

    @override
    async def select_next_agent(self, chat_history: ChatHistory, participant_descriptions: dict[str, str],) -> StringResult:
        # Ask the mediator to select the next participant
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.selection_prompt,
                    KernelArguments(
                        topic=self.topic,
                        participants="\n".join([f"{k}: {v}" for k, v in participant_descriptions.items()]),
                    ),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Now select the next participant to speak."),
        )
        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )
        participant_name_with_reason = StringResult.model_validate_json(response.content)
        print("*********************")
        print(f"Next participant: {participant_name_with_reason.result}\nReason: {participant_name_with_reason.reason}.")
        print("*********************")
        if participant_name_with_reason.result in participant_descriptions:
            return participant_name_with_reason
        raise RuntimeError(f"Unknown participant selected: {response.content}.")

    @override
    async def filter_results(self, chat_history: ChatHistory,) -> MessageResult:
        # Summarize the discussion at the end
        if not chat_history.messages:
            raise RuntimeError("No messages in the chat history.")
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.result_filter_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Please summarize the discussion."),
        )
        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )
        string_with_reason = StringResult.model_validate_json(response.content)
        return MessageResult(
            result=ChatMessageContent(role=AuthorRole.ASSISTANT, content=string_with_reason.result),
            reason=string_with_reason.reason,
        )

## 5. Define the Agent Response Callback

This function is called whenever an agent produces a message. It allows you to capture or display agent responses in real time.

In [8]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Print agent responses as they are produced."""
    print(f"**{message.name}**\n{message.content}")

## 6. Orchestrate the Group Chat

Now we bring everything together: create the agents, the manager, and the runtime, then start the group chat orchestration. The discussion will proceed automatically, with the manager controlling the flow.

In [9]:
async def main():
    """Run the multi-agent group chat orchestration."""
    agents = get_agents()
    group_chat_orchestration = GroupChatOrchestration(
        members=agents,
        manager=ChatCompletionGroupChatManager(
            topic="What does a good life mean to you personally?",
            service=get_azure_chat_completion(),
            max_rounds=10,
        ),
        agent_response_callback=agent_response_callback,
    )
    runtime = InProcessRuntime()
    runtime.start()
    orchestration_result = await group_chat_orchestration.invoke(
        task="Please start the discussion.",
        runtime=runtime,
    )
    value = await orchestration_result.get()
    print(value)
    await runtime.stop_when_idle()

## 7. Run the Orchestration

Uncomment and run the following cell to start the group chat orchestration. You should see the discussion unfold, with the manager selecting participants and summarizing at the end.

In [10]:
# Uncomment to run the orchestration
# import nest_asyncio
# nest_asyncio.apply()
# asyncio.run(main())

## Exercise Complete!

You have now built and run a multi-agent group chat using the new Semantic Kernel agent orchestration API. Try modifying the agent instructions, prompts, or the topic to see how the discussion changes.